pull cifs from materials project using code from chemeleon

remove duplicates and whatever using code from chemeleon

preprocess cifs using code from crystallm

tokenize cifs using code from crystallm

# Code from Chemeleon - installed all requirements from chemeleon

In [27]:
import os
import re
import random
from pathlib import Path
import time
import dotenv
import warnings
import datetime

from tqdm import tqdm
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from mp_api.client import MPRester
from pymatgen.core import Structure
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from ase.visualize import view

warnings.filterwarnings("ignore")

In [28]:
dotenv.load_dotenv()
MP_API_KEY = os.getenv("MP_API_KEY")

In [29]:
with MPRester(MP_API_KEY) as mpr:
    docs = mpr.summary.search(
        num_sites=[0, 40],
        energy_above_hull=[0, 0.25],
        theoretical=False,
        fields=[
            "material_id",
            "structure",
            "energy_above_hull",
            "band_gap",
            "theoretical",
        ],
    )

Retrieving SummaryDoc documents: 100%|██████████| 33476/33476 [00:56<00:00, 588.26it/s]


In [30]:
excluded_gas_list = [
    "H",
    "He",
    "N",
    "O",
    "F",
    "Ne",
    "Cl",
    "Ar",
    "Kr",
    "Xe",
    "Rn",
    "Fr",
    "Og",
]

In [39]:
data = []
for doc in tqdm(docs):
    st = doc.structure
    elements = [elmt.symbol for elmt in st.composition.elements]

    if len(elements) == 1 and elements[0] in excluded_gas_list:
        # print(elements)
        continue

    if max(st.lattice.abc) > 20:
        # print(st.formula, st.lattice.abc)
        continue

    row = {
        "material_id": doc.material_id,
        "cif": st.to(fmt="cif"),
    }
    data.append(row)

100%|██████████| 33476/33476 [00:48<00:00, 684.69it/s]


In [40]:
df_mp_api = pd.DataFrame(data)

In [41]:
df_mp_api = df_mp_api.drop_duplicates(subset="material_id")
# shuffle
df_mp_api = df_mp_api.sample(frac=1, random_state=42).reset_index()
df_train = df_mp_api[0:1000].drop(columns=['Unnamed: 0', 'index'], errors='ignore')
df_test = df_mp_api[1000:2000].drop(columns=['Unnamed: 0', 'index'], errors='ignore')
df_val= df_mp_api[2000:3000].drop(columns=['Unnamed: 0', 'index'], errors='ignore')

In [42]:
import pickle
import gzip

In [55]:
train_tmp = list(df_train.itertuples(index=False, name=None))
test_tmp = list(df_test.itertuples(index=False, name=None))
val_tmp = list(df_val.itertuples(index=False, name=None))

In [56]:
train = [(f"MP_{str(item[0])}", item[1]) for item in train_tmp]
test = [(f"MP_{str(item[0])}", item[1]) for item in test_tmp]
val = [(f"MP_{str(item[0])}", item[1]) for item in val_tmp]

In [60]:
with gzip.open("train_cifs_1000.pkl.gz", "wb") as f:
    pickle.dump(train, f)
with gzip.open("test_cifs_1000.pkl.gz", "wb") as f:
    pickle.dump(test, f)
with gzip.open("val_cifs_1000.pkl.gz", "wb") as f:
    pickle.dump(val, f)